# **Setup the development framework such that changes in py files auto load into this file**
- You can ignore this if your interest is only in the business logic and design. 
- The steps in this section are only environment set related

### **Make sure py file change are loaded automatically**

In [ ]:
%load_ext autoreload
%autoreload 2

### **Import basic package required for this assignment**

In [ ]:
import os
import pandas as pd
import db.constants as C
import numpy as np


### **Make sure right sources are getting picked**

In [ ]:
from dao.base_transformer import BaseDBTransformer

### **Make sure the latest class are getting loaded.**

In [ ]:
# 1. Import the modules first (so Python knows them)
import importlib
import service.product_transformer
import service.cart_transformer
import service.purchase_transformer
import service.discount_transformer
import db.constants
import db.dbsql

# 2. Reload them (useful after edits)
importlib.reload(service.product_transformer)
importlib.reload(service.cart_transformer)
importlib.reload(service.purchase_transformer)
importlib.reload(service.discount_transformer)
importlib.reload(db.constants)

# 3. Import the classes fresh from the reloaded modules
from service.product_transformer import ProductTransformer
from service.cart_transformer import CartTransformer
from service.purchase_transformer import PurchaseTransformer
from service.discount_transformer import DiscountTransformer
import db.constants
import random
from db.dbsql import SessionLocal, Product, ProductClass

# **Transformers**
- This section is the core design of the project
- There are four transformers 
   - Products transformer: This is the database of the list of the products in EStore. The prodcuts.csv file for demo has 9 products. 
   - Cart transformer: This is user selected products added into the cart instance specific to the user. 
   - purchase transfromer: This is checkout functionaility of the cart. 
        - It has two kinds of purchases. With discount by applying discount code.
        - It also supports purchase without applying discount code. If no discount is applied it generates a ne discount code and adds to discounts for the subsequent purchases. 
   - Discount transformer: This handles the discounts codes after the purchase. 

   - For each these transforers there is .py  file and .csv file in the src directory. 
   - Each transfomer derives from the base transfomer class which manages the csv read/write for that transformer. 
   - The base transformer class also handles basis CRUD operation if they are at the level of the dataframe(entire row level operations) without having to deal with individual columns (as columns are specific to the transformer)

- The code is organized as a typical pyproject.toml file. 

## **Product Transformer - demonstration**

#### **Get the product**

In [ ]:
pt = ProductTransformer()
prds_all = pt.list_all_products()
prds_all.head(5)

In [ ]:
prds_2050 = pt.list_products_in_class_df(2050)
prds_2050.to_dict(orient='records')

#### **Get a single product**

In [ ]:
df = pt.list_all_products(5)
print(type(df))
df

#### **Get the product classes and description**

In [ ]:
prod_classes = pt.get_products_class()

for i,pclass in enumerate(prod_classes[C.ptyp]):
    print("Iteration i ", i, " p class code = ", pclass)
    if ( i>3 ):
        break
    pc_dict = pt.list_products_in_class(pclass)
    print(pc_dict)

prod_classes


## **Discount Transformer - demonstration**

In [ ]:
dt = DiscountTransformer()
dis_all = dt.list_all_codes()
dis_all

## **Purchase Transformer - demonstration**

In [ ]:
prt = PurchaseTransformer()
prt.list_orders_by_customer(2)

## **Cart Transformer - demonstration**

In [ ]:
ct = CartTransformer
ct.list_all()

# **Business logic - Helper functions**

### **Get a random selection of 5 products to purchase**

In [ ]:
products_to_buy = pt.getRandomProducts(5)
customer_id=3
products_to_buy


### **Add these products to the cart of a selected customer**

##### **Select the items**

In [ ]:
selected_items = products_to_buy.loc[:, [C.pid]]
selected_items[C.qnt] = np.random.randint(1, 3, size=len(selected_items))
items = selected_items.to_dict(orient='records')
items


##### **Add Them to the cart**

In [ ]:
CartTransformer.addToCart(customer_id, items, debug=True)

##### **Verify the addition**

In [ ]:
CartTransformer.list_cust(3)
BaseDBTransformer.read(C.cart)

### **Purchase the items in the cart**

In [ ]:
order_id = prt.purchase_cart_items(customer_id, debug=True)
print("Order Id =", order_id)


### **Purchase cart items with discount coupon**

In [ ]:
customer_id = 3
cust_cart = BaseDBTransformer.read(C.cart, **{C.custid: customer_id})
for i, dict in enumerate(cust_cart.to_dict(orient='records')):
    prd_df = BaseDBTransformer.read(C.prd, dict[C.pid])
    prd_dict = prd_df.iloc[0].to_dict()
    print(prd_dict)
    print(cust_cart)


In [ ]:
discount_id = str(customer_id) + "__" + str("10072")
print(discount_id)
res_with_d = prt.purchase_discounted_cart_items(customer_id, discount_id, debug=True)
print( " Order Id with discount =", res_with_d)


In [ ]:
BaseDBTransformer.read(C.discounts)
BaseDBTransformer.read(C.discounts, discount_id)

# **Admin APIs**

### **Get all discount codes**

In [ ]:
all_disc_codes = dt.list_all_codes()
all_disc_codes


### **Get codes eligible - But not enabled**

In [ ]:
eligible_disc_codes = dt.list_eligible_codes(customer_id)
eligible_disc_codes

### **Get all active codes - But not yet used by the customer**

In [ ]:
active_disc_codes = dt.list_active_codes()
active_disc_codes

### **Enable a discount code for a customer**

#### **First change the status**

In [ ]:
# discount id is custoemrid + __ + Order_id
discount_id = "3__10072" 
dt.enable_discount_codes(discount_id, 10, "NEW_CUSTOMER")

#### **Second verify the record that status has really changed**

In [ ]:
active_disc_codes = dt.list_active_codes()
active_disc_codes

### **Test the FASTAPIs**

In [ ]:
df = ProductTransformer.get_products_class()
mapped = df.to_dict(orient="records")
mapped